In [41]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0.5,
    base_url="http://localhost:11434",
    stream_usage=True  # 流模式需开启统计
)

这个是0.3版本的memory，主要是把chain换成了langgraph的形式，看输入抽象类一个是差不多的

In [42]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

当我们运行应用程序时，我们传入一个dict指定 的配置thread_id。此 ID 用于区分对话线程（例如，在不同用户之间）。

In [43]:
config = {"configurable": {"thread_id": "abc123"}}

然后我们可以调用该应用程序：

In [44]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

<think>

</think>

Hello, Bob! How can I assist you today? 😊


In [45]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>

</think>

Greetings! My name is Bob. I'm an AI assistant created to provide helpful and harmless responses.


In [47]:
query = "What's my name?"
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>

</think>

Hi! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I don't have access to personal information about users. For any inquiries, please go directly to DeepSeek-R1.


In [48]:
#langchain 1.0.0 被remove了，比较简单的一个实现形式但是用不了

from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

history = ConversationBufferMemory()
history.save_context({"input": "你好啊"}, {"output": "你也好啊"})
print(history.load_memory_variables({}))

history.save_context({"input": "你再好啊"}, {"output": "你又好啊"})
print(history.load_memory_variables({}))

history.chat_memory.add_user_message("你在干嘛")
history.chat_memory.add_ai_message("我在学习")
print(history.load_memory_variables({}))


{'history': 'Human: 你好啊\nAI: 你也好啊'}
{'history': 'Human: 你好啊\nAI: 你也好啊\nHuman: 你再好啊\nAI: 你又好啊'}
{'history': 'Human: 你好啊\nAI: 你也好啊\nHuman: 你再好啊\nAI: 你又好啊\nHuman: 你在干嘛\nAI: 我在学习'}
